In [18]:
from pymongo import MongoClient
from itertools import chain
import re


def cleanhtml(raw_html):
    cleantext = raw_html
    if isinstance(raw_html, str):
        cleanr = re.compile('<.*a>')
        cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleandate(raw_html):
    cleaneddate = raw_html
    if isinstance(raw_html, str):
        cleandt = re.compile(r'([0-3][0-9])\.([0-9]{2})\.([0-9]{4})')
        cleaneddate = re.sub(cleandt, '', raw_html)
    return cleaneddate


client = MongoClient("mongodb://localhost:27017/")
db = client['ds_shipment_tracking']
dhl_shipment_events = db.dhl_shipment_events

res = dhl_shipment_events.find()


def cleanstatussen(res):
    t = list(map(lambda x: list(map(lambda y: y.get('status'), x.get('events'))), res))
    c = list(chain.from_iterable(t))#stati
    f = list(map(lambda x: cleanhtml(x), c))
    cdt = list(map(lambda x: cleandate(x), f))
    #s = set(cdt)

    #cleanlist = []
    counts = {}
    for status in cdt:
        if status in counts:
            counts[status] = counts[status]+1
        else:
            counts[status] =1
    for k, v in counts.items():
        counts[k] = [counts[k]]#for pandas

    return counts


    '''
        if status is not None:
            cleanlist.append(i)

    sortedcl = sorted(cleanlist)

    countlist = ((i, sortedcl.count(i)) for i in sortedcl)
    number_of_status = set(countlist)
    #print(number_of_status)
    '''

'''
    for i in number_of_status:
        print(i)
        print('\n')
'''

'''   
    with open('Alle_Statis.txt', 'w') as f:
        for i in sortedcl:
            f.write("%s\n\n" % i)
'''

counts = cleanstatussen(res)




{'Die Sendung wurde elektronisch angeku00FCndigt. Sobald die Sendung von uns bearbeitet wurde, erhalten Sie weitere Informationen.': 60610, 'Die Sendung wurde von DHL bearbeitet und wird fu00FCr den Weitertransport in die Region des Empfu00E4ngers vorbereitet.': 56849, 'Die Sendung ist in der Region des Empfu00E4ngers angekommen und wird im nu00E4chsten Schritt zur Zustellbasis transportiert.': 58843, 'Die Sendung wurde in das Zustellfahrzeug geladen.': 60958, 'Die Sendung wurde erfolgreich zugestellt.': 48703, 'Die Sendung befindet sich auf dem Weg zur Packstation.': 1640, 'Die Sendung liegt in der  zur Abholung bereit.': 1667, 'Der Empfu00E4nger hat die Sendung aus der PACKSTATION abgeholt.': 1643, 'Die Sendung konnte beim 2. Zustellversuch nicht zugestellt werden.': 25, 'Die Sendung konnte nicht zugestellt werden und wird in die Filiale gebracht. Fu00FCr die Abholung ist neben der Benachrichtigung die Vorlage Ihres Ausweises sowie ggf. eine Vollmacht erforderlich.': 7963, 'Die Sendu

In [25]:
import pandas as pd
%matplotlib inline


In [36]:
df = pd.DataFrame.from_dict(counts).transpose()


In [33]:
df.to_excel("AnzahlStatis.xlsx")